In [1]:
!pip install pandas==1.3.4
!pip install transformers==4.12.5
!pip install datasets==1.15.1
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import os
import pickle

from collections import Counter

import pandas as pd
from sklearn.metrics import classification_report

import numpy as np
import torch
import torch.nn as nn

import transformers
from transformers import Trainer
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers.data.data_collator import DataCollatorWithPadding

import datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import load_metric

## Global variables

In [3]:
DATA_FOLDER = '/notebooks/Data/bert_sequence_classification'
DATA_FILE = '/notebooks/KURI-BERT/notebooks/full_formula_w_fts/Stance_Recognition_Task/pe_dataset_for_bert_finetune_stance_recog.pt'
RESULTS_FOLDER = '/notebooks/Results/bert_sequence_classification'
# dataset_df = pd.read_pickle("/notebooks/KURI-BERT/data/pe_dataset_w_essay_position_pos_tags_pickle")

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cuda')

## Load data

In [6]:
dataset = torch.load(DATA_FILE)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_x', 'label_ComponentType', 'relation_SupportAttack', 'labels', 'label_LinkedNotLinked', 'split', 'essay', 'argument_bound_1', 'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr', 'total_paras', 'token_count', 'token_count_covering_para', 'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count', 'succeeding_tokens_in_sentence_count', 'token_ratio', 'relative_position_in_para_char', 'is_in_intro', 'relative_position_in_para_token', 'is_in_conclusion', 'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para', 'nr_following_comps_in_para', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'para_ratio', 'first_or_last', 'strct_fts_w_position_in_essay', 'component_pos_tags', 'strct_fts_essay_position_pos_tags'],
        num_rows: 3300
    })
    test: Dataset({
        features: ['essay_nr', 'component_id', 'label_and_

In [8]:
dataset['train']['structural_fts_as_text_combined'][230]

'Topic: Schools should only teach children the academic subject, Sentence: Although those above reasons are acceptable, I would argue that there are many things much more important than having a job, Structural Features: 3, Yes, No, No, No'

In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [10]:
label_names = set(dataset['train']['labels'])
label_nb = len(label_names)
labels = ClassLabel(num_classes=label_nb, names=label_names)

In [11]:
labels

ClassLabel(num_classes=2, names={'Support', 'Attack'}, names_file=None, id=None)

In [12]:
def tokenize(batch):
    tokens = tokenizer(batch['structural_fts_as_text_combined'], truncation=True, padding=True, max_length=512)
    tokens['labels'] = labels.str2int(batch['labels'])
    return tokens

# this is just the text. if the results are nice, check transfer with text + topic 

In [13]:
dataset = dataset.map(tokenize, batched=True)

Parameter 'function'=<function tokenize at 0x7fb4870e5790> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['argument_bound_1', 'argument_bound_2', 'argument_id', 'attention_mask', 'component_id', 'component_pos_tags', 'essay', 'essay_nr', 'first_or_last', 'input_ids', 'is_first_in_para', 'is_in_conclusion', 'is_in_intro', 'is_last_in_para', 'label_ComponentType', 'label_LinkedNotLinked', 'label_and_comp_idxs', 'label_x', 'labels', 'nr_following_comps_in_para', 'nr_preceeding_comps_in_para', 'para_nr', 'para_ratio', 'paragraph', 'preceeding_tokens_in_sentence_count', 'relation_SupportAttack', 'relative_position_in_para_char', 'relative_position_in_para_token', 'sentence', 'split', 'strct_fts_essay_position_pos_tags', 'strct_fts_w_position_in_essay', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'succeeding_tokens_in_sentence_count', 'text', 'token_count', 'token_count_covering_para', 'token_ratio', 'token_type_ids', 'tokens_count_covering_sentence', 'total_paras'],
        num_rows: 3300
    })
    test: Dataset({
        f

In [16]:
dataset['train']['input_ids'][0:4]

[tensor([  101,  8476,  1024,  2974, 18708,  2015,  2336,  1005,  1055, 14842,
          2612,  1997, 22031,  2009,  1010,  6251,  1024,  2174,  1010,  2007,
          3811,  3935,  2974,  1010,  2336,  2024,  2583,  2000,  2131,  2592,
          2013,  3097,  3032,  1998,  2130, 10639,  2007,  3097,  2814,  1010,
          8332,  2838,  1024,  1017,  1010,  2053,  1010,  2053,  1010,  2053,
          1010,  2053,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]),
 tensor([  101,  8476,  1024,  1000,  7279,  1998,  3259,  1000,  7749,  1010,
          6251,  1024,  2027,  2024,  2036,  4659, 22979,  1025,  2367,  4127,
          1997, 11360,  3980,  1010,  2005,  2742,  1010,  3674,  1011,  3601,
          398

In [17]:
train_dataset = dataset['train'].shuffle(seed=42)
test_dataset = dataset['test'].shuffle(seed=42)

train_val_datasets = dataset['train'].train_test_split(train_size=0.8, seed=42)
train_dataset = train_val_datasets['train']
val_dataset = train_val_datasets['test']

In [18]:
dataset_d = {}
dataset_d['train'] = train_dataset
dataset_d['test'] = test_dataset
dataset_d['val'] = val_dataset

In [19]:
tokenizer.decode(dataset['train'][2945]['input_ids'])

'[CLS] topic : study at school or get a job?, sentence : there are many ways that can train children to learn independent and social skills instead of getting a job., structural features : 4, no, yes, no, yes [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [20]:
# sanity check
set(dataset_d['train']['split'])

{'TRAIN'}

In [21]:
# sanity check
set(dataset_d['val']['split'])

{'TRAIN'}

In [22]:
# sanity check
set(dataset_d['test']['split'])

{'TEST'}

In [23]:
# global variables
NUM_LABELS = labels.num_classes
BATCH_SIZE = 48
NB_EPOCHS = 10

In [24]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=NUM_LABELS)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [25]:
counter = Counter(dataset_d['train']['labels'].tolist())
counter

Counter({0: 2376, 1: 264})

In [26]:
class_weights = [max(counter.values()) / counter[k] for k in sorted(counter.keys())]
class_weights = torch.FloatTensor(class_weights).to(device)
class_weights

tensor([1., 9.], device='cuda:0')

In [27]:
# https://huggingface.co/transformers/main_classes/trainer.html
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.CrossEntropyLoss()#weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [28]:
metric = load_metric('f1')

def compute_metrics(eval_pred):
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    
    return metric.compute(predictions=predictions, references=labels, average='macro')

In [29]:
training_args = TrainingArguments(
    
    # output
    output_dir=RESULTS_FOLDER,          
    
    # params
    num_train_epochs=NB_EPOCHS,               # nb of epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # cf. paper Sun et al.
    learning_rate=1e-5,#2e-5,                 # cf. paper Sun et al.
#     warmup_steps=500,                         # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                         # cf. paper Sun et al.
    weight_decay=0.01,                        # strength of weight decay
    
    # eval
    evaluation_strategy="steps",              # cf. paper Sun et al.
    eval_steps=20,                            # cf. paper Sun et al.
    
    # log
    logging_dir="/notebooks/Results/bert_sequence_classification/tb_logs",  
    logging_strategy='steps',
    logging_steps=20,
    
    # save
    save_strategy='steps',
    save_total_limit=2,
    # save_steps=20, # default 500
    load_best_model_at_end=True,              # cf. paper Sun et al.
    # metric_for_best_model='eval_loss' 
    metric_for_best_model='f1'
)

In [30]:
trainer = CustomTrainer( # Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [31]:
results = trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: relative_position_in_para_token, component_pos_tags, is_last_in_para, is_in_conclusion, token_count, nr_following_comps_in_para, relation_SupportAttack, is_in_intro, token_count_covering_para, label_ComponentType, tokens_count_covering_sentence, first_or_last, para_nr, label_x, structural_fts_as_text, structural_fts_as_text_combined, essay, is_first_in_para, paragraph, argument_bound_1, strct_fts_essay_position_pos_tags, relative_position_in_para_char, nr_preceeding_comps_in_para, token_ratio, preceeding_tokens_in_sentence_count, essay_nr, text, argument_bound_2, para_ratio, succeeding_tokens_in_sentence_count, label_and_comp_idxs, component_id, strct_fts_w_position_in_essay, split, total_paras, label_LinkedNotLinked, sentence, argument_id.
***** Running training *****
  Num examples = 2640
  Num Epochs = 10
  Instantaneous batch size per devi

Step,Training Loss,Validation Loss,F1
20,0.602800,0.442553,0.478673
40,0.367700,0.285541,0.478673
60,0.330200,0.281426,0.478673
80,0.315300,0.272258,0.478673
100,0.293700,0.285125,0.478673
120,0.333500,0.267297,0.478673
140,0.266300,0.265372,0.548883
160,0.275100,0.247365,0.556628
180,0.225500,0.243141,0.575773
200,0.226200,0.238090,0.692768


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: relative_position_in_para_token, component_pos_tags, is_last_in_para, is_in_conclusion, token_count, nr_following_comps_in_para, relation_SupportAttack, is_in_intro, token_count_covering_para, label_ComponentType, tokens_count_covering_sentence, first_or_last, para_nr, label_x, structural_fts_as_text, structural_fts_as_text_combined, essay, is_first_in_para, paragraph, argument_bound_1, strct_fts_essay_position_pos_tags, relative_position_in_para_char, nr_preceeding_comps_in_para, token_ratio, preceeding_tokens_in_sentence_count, essay_nr, text, argument_bound_2, para_ratio, succeeding_tokens_in_sentence_count, label_and_comp_idxs, component_id, strct_fts_w_position_in_essay, split, total_paras, label_LinkedNotLinked, sentence, argument_id.
***** Running Evaluation *****
  Num examples = 660
  Batch size = 48
The following columns in the eva

In [32]:
# save best model
#trainer.save_model(os.path.join("/notebooks/KURI-BERT/notebooks/full_formula_w_fts/Stance_Recognition_Task", 'best-model-for-stance_recognition.pth'))
torch.save(model.state_dict(), '/notebooks/KURI-BERT/notebooks/full_formula_w_fts/icann_finetuned_work/stance_rec_finetuned_model_new_single.pth')

In [33]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [34]:
test_trainer = Trainer(model, data_collator=DataCollatorWithPadding(tokenizer))
test_raw_preds, test_labels, _ = test_trainer.predict(test_dataset)
test_preds = np.argmax(test_raw_preds, axis=1)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: relative_position_in_para_token, component_pos_tags, is_last_in_para, is_in_conclusion, token_count, nr_following_comps_in_para, relation_SupportAttack, is_in_intro, token_count_covering_para, label_ComponentType, tokens_count_covering_sentence, first_or_last, para_nr, label_x, structural_fts_as_text, structural_fts_as_text_combined, essay, is_first_in_para, paragraph, argument_bound_1, strct_fts_essay_position_pos_tags, relative_position_in_para_char, nr_preceeding_comps_in_para, token_ratio, p

In [35]:
sum(test_preds)

69

In [36]:
target_name = labels.int2str([0,1])
print(classification_report(test_labels, test_preds, target_names=target_name, digits=3))

              precision    recall  f1-score   support

     Support      0.948     0.969     0.958      1016
      Attack      0.536     0.407     0.463        91

    accuracy                          0.922      1107
   macro avg      0.742     0.688     0.710      1107
weighted avg      0.914     0.922     0.917      1107

